# Custom solvers
In this example, we show how to define custom solvers. Our system
will again be silicon, because we are not very imaginative

In [1]:
using DFTK, LinearAlgebra

a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

# We take very (very) crude parameters
model = model_LDA(lattice, atoms)
kgrid = [1, 1, 1]
Ecut = 5
basis = PlaneWaveBasis(model, Ecut; kgrid=kgrid);

We define our custom fix-point solver: simply a damped fixed-point

In [2]:
function my_fp_solver(f, x0, max_iter; tol)
    mixing_factor = .7
    x = x0
    fx = f(x)
    for n = 1:max_iter
        inc = fx - x
        if norm(inc) < tol
            break
        end
        x = x + mixing_factor * inc
        fx = f(x)
    end
    (fixpoint=x, converged=norm(fx-x) < tol)
end;

Our eigenvalue solver just forms the dense matrix and diagonalizes
it explicitly (this only works for very small systems)

In [3]:
function my_eig_solver(A, X0; maxiter, tol, kwargs...)
    n = size(X0, 2)
    A = Array(A)
    E = eigen(A)
    λ = E.values[1:n]
    X = E.vectors[:, 1:n]
    (λ=λ, X=X, residual_norms=[], iterations=0, converged=true, n_matvec=0)
end;

Finally we also define our custom mixing scheme. It will be a mixture
of simple mixing (for the first 2 steps) and than default to Kerker mixing.
In the mixing interface `δF` is ``(ρ_\text{out} - ρ_\text{in})``, i.e.
the difference in density between two subsequent SCF steps and the `mix`
function returns ``δρ``, which is added to ``ρ_\text{in}`` to yield ``ρ_\text{next}``,
the density for the next SCF step.

In [4]:
struct MyMixing
    α  # Damping parameter
end
MyMixing() = MyMixing(0.7)

function DFTK.mix(mixing::MyMixing, basis, δF::RealFourierArray, δF_spin=nothing; n_iter, kwargs...)
    if n_iter <= 2
        # Just do simple mixing on total density and spin density (if it exists)
        (mixing.α * δF, isnothing(δF_spin) ? nothing : mixing.α * δF_spin)
    else
        # Use the KerkerMixing from DFTK
        DFTK.mix(KerkerMixing(α=mixing.α), basis, δF, δF_spin; kwargs...)
    end
end

That's it! Now we just run the SCF with these solvers

In [5]:
scfres = self_consistent_field(basis;
                               tol=1e-8,
                               solver=my_fp_solver,
                               eigensolver=my_eig_solver,
                               mixing=MyMixing());

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   Diag
---   ---------------   ---------   --------   ----
  1   -7.200034051584         NaN   3.43e-01    0.0 
  2   -7.242360972977   -4.23e-02   1.66e-01    0.0 
  3   -7.248323239800   -5.96e-03   7.24e-02    0.0 
  4   -7.248939855828   -6.17e-04   3.97e-02    0.0 
  5   -7.249130570912   -1.91e-04   2.20e-02    0.0 
  6   -7.249189473889   -5.89e-05   1.23e-02    0.0 
  7   -7.249207938905   -1.85e-05   7.01e-03    0.0 
  8   -7.249213872539   -5.93e-06   4.04e-03    0.0 
  9   -7.249215839064   -1.97e-06   2.37e-03    0.0 
 10   -7.249216513283   -6.74e-07   1.41e-03    0.0 
 11   -7.249216752480   -2.39e-07   8.47e-04    0.0 
 12   -7.249216840125   -8.76e-08   5.17e-04    0.0 
 13   -7.249216873178   -3.31e-08   3.19e-04    0.0 
 14   -7.249216885954   -1.28e-08   1.98e-04    0.0 
 15   -7.249216890992   -5.04e-09   1.24e-04    0.0 


Note that the default convergence criterion is on the difference of
energy from one step to the other; when this gets below `tol`, the
"driver" `self_consistent_field` artificially makes the fixpoint
solver think it's converged by forcing `f(x) = x`. You can customize
this with the `is_converged` keyword argument to
`self_consistent_field`.